In [ ]:
import numpy as np
import math

In [ ]:
l = 1

In [ ]:
def expwts(X):
    for i in range(len(X)):
      if math.isnan(X[i]):
        X[i]=0

    S = sum([math.exp(X[i]) for i in range(len(X))])
    return np.array([[math.exp(X[i])/S] for i in range(len(X))])

def srect(b,x):
    if x<=0:
        return 0
    if 0<b<x:
        return b*b/(2*x)
    else:
        return b-x/2

def chop(a,b):
    if b ==0:
        return 0
    else:
        return max(0,min(1,a/b))

def Xlogx(t):
    if t<= 0 :
        return 0
    else:
        return t*np.log(t)



def max_X(w, Y, A, a):
    w_1 = A.T@w
    Z = a*Y*Y/(2*w)
    Z_1 = A.T@Z
    X = np.array([expwts(-Z_1[i]/(3*a*w_1[i][0])).reshape((len(Z_1[i]))) for i in range(len(Z_1)-len(Z_1[0]))])
    return X

def max_wY(B,X,A,a,b):
    n = len(B)
    m = len(X)
    k = len(B[0])
    X_1 = A@X
    S = np.array([[sum([Xlogx(X[e][j]) for j in range(k)])] for e in range(m)])
    S_1 = A@S
    b_1 = np.array([[sum([srect(B[v][j],a*X_1[v][j]) for j in range(k)])] for v in range(n)])
    w = expwts((b_1-3*a*S_1)/(a*b))
    Y = np.array([[w[v][0]*chop(B[v][j],a*X_1[v][j]) for j in range(k)]for v in range(n)])
    return w,Y

In [ ]:


def Minimizer(B,A,epsilon):
    a = 18
    b = 27*(2+np.log(len(B[0])))
    X = np.ones((len(A[0]) -len(B[0]),len(B[0])))/len(B[0])
    for i in range(int(20*np.log(2/epsilon))):
        X = np.vstack([X,np.eye(len(B[0]))])
        w,Y = max_wY(B,X,A,a,b)
        X = max_X(w,Y,A,a)
    X = np.vstack([X,np.eye(len(B[0]))])
    return w,Y,X

In [ ]:
def final(D,B,epsilon):
    C = np.eye(len(D[0]))
    B_1 = np.hstack([-B, B])
    B_final = np.hstack([B_1,np.zeros((len(B),1))])
    A_final = np.hstack([-D,D])
    A = np.hstack([A_final,B_final])
    print(A)
    n = len(A_final)
    k = len(B_final[0])
    m = len(A_final[0])
    print(n,m,k)
    A_pos = abs(A)
    t = int(10*np.log(n)*(np.log(2+k))**2)
    w = np.ones((n,1))/n
    Y = np.zeros((n,k))
    X = np.ones((m,k))/m
    X = np.vstack([X,np.eye(k)])
    for i in range(t):
      Jz_w,Jz_y,Jz_x = np.zeros((n,1)),A@X,-A.T@Y
      w_1,Y_1,X_1 = Minimizer(Jz_y,A_pos,epsilon)
      w_2,Y_2,X_2 = np.zeros((n,1)), 2*A@X_1, -2*A.T@Y_1
      S = Minimizer(Jz_y+Y_2,A_pos,epsilon)
      #print(S[0],S[1],S[2])
      w+=S[0]
      Y+=S[1]
      X+=S[2]
    return w/t,Y/t,X/t




In [ ]:
D = np.array([[1,1,0,0],[-1,0,-1,0],[0,0,1,1],[0,-1,0,-1]])
B = np.array([[1,0],[-1,0],[0,1],[0,-1]])
final(D,B,0.01)

[[-1. -1.  0.  0.  1.  1.  0.  0. -1.  0.  1.  0.  0.]
 [ 1.  0.  1.  0. -1.  0. -1.  0.  1.  0. -1.  0.  0.]
 [ 0.  0. -1. -1.  0.  0.  1.  1.  0. -1.  0.  1.  0.]
 [ 0.  1.  0.  1.  0. -1.  0. -1.  0.  1.  0. -1.  0.]]
4 8 5


<ipython-input-3-8d931aa675cf>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if math.isnan(X[i]):
<ipython-input-3-8d931aa675cf>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  S = sum([math.exp(X[i]) for i in range(len(X))])
<ipython-input-3-8d931aa675cf>:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return np.array([[math.exp(X[i])/S] for i in range(len(X))])


(array([[0.2548192 ],
        [0.25489672],
        [0.25461719],
        [0.25489765]]),
 array([[0.00000000e+00, 3.79689452e-18, 1.84584411e-01, 1.72842200e-18,
         1.72842200e-18],
        [1.84588075e-01, 1.73076992e-18, 0.00000000e+00, 3.79432431e-18,
         1.73076992e-18],
        [3.38346094e-18, 0.00000000e+00, 5.60222312e-19, 1.84383675e-01,
         5.60222312e-19],
        [2.32441611e-18, 1.84578132e-01, 4.56440192e-18, 0.00000000e+00,
         2.32441611e-18]]),
 array([[0.19607267, 0.20662496, 0.19607167, 0.20662496, 0.20662496],
        [0.20662496, 0.19607318, 0.19607117, 0.20662496, 0.20662496],
        [0.19606632, 0.20662496, 0.20662496, 0.19607802, 0.20662496],
        [0.20662496, 0.19606683, 0.20662496, 0.19607752, 0.20662496],
        [0.19607267, 0.20662496, 0.19607167, 0.20662496, 0.20662496],
        [0.20662496, 0.19607318, 0.19607117, 0.20662496, 0.20662496],
        [0.19606632, 0.20662496, 0.20662496, 0.19607802, 0.20662496],
        [0.20662496, 0